<a href="https://colab.research.google.com/github/tmyc131/lrc_file_crawler/blob/master/kanana_1_5_2_1b_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -U transformers

Model page: https://huggingface.co/kakaocorp/kanana-1.5-2.1b-base

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/kakaocorp/kanana-1.5-2.1b-base)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="kakaocorp/kanana-1.5-2.1b-base")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'Name of the program (e.g., Microsoft Word), or a description of its function.\n\nName of the program (e.g., Word) or a description of its function.\n\nName (e.g., Google Chrome), or a description of its function (e.g., web browser).\n\nName (e.g., Google Chrome) or a description of its function (e.g., web browser).\n\nName (e.g., Netflix)\n\nName (e.g., Netflix)\n\nName (e.g., Netflix) or a description of its function (e.g., streaming service).\n\nName (e.g., Netflix) or a description of its function (e.g., streaming service).\n\nName (e.g., Netflix) or a description of its function (e.g., streaming service).\n\nName (e.g., Netflix) or a description of its function (e.g., streaming service).\n\nName (e.g., Netflix) or a description of its function (e.g., streaming service).\n\nName (e.g., Netflix) or a description of its function (e.g., streaming service).\n\nName (e.g., Netflix) 

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")

In [ ]:
# requirements.txt
transformers>=4.30.0
torch>=2.0.0
pandas>=1.5.0
openpyxl>=3.1.0
python-docx>=0.8.11
PyQt5>=5.15.0
scikit-learn>=1.3.0
numpy>=1.24.0
sentence-transformers>=2.2.0


In [2]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.4 MB/s eta 0:00:00


In [4]:
pip install PyQt5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 22.6 MB/s eta 0:00:00


In [5]:
# 기본 라이브러리 임포트
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pickle
from pathlib import Path

# AI 모델 관련
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 문서 처리
from docx import Document
from docx.shared import Inches
import openpyxl

# GUI
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *


In [6]:
class DocumentProcessor:
    def __init__(self):
        self.checklist_structure = self._load_checklist_structure()
        self.review_patterns = self._load_review_patterns()

    def _load_checklist_structure(self):
        """체크리스트 구조 정의"""
        return {
            "자체점검_제출항목": [
                "자체점검_결과서_작성여부",
                "자체점검_결과_개선내역서_작성여부",
                "화학사고예방관리계획서_변경내역_관리대장_작성여부"
            ],
            "사업장_내부활동": {
                "화학사고예방관리계획서_이해도": [
                    "사업주_이해도",
                    "취급자_이해도",
                    "안전환경담당자_이해도",
                    "방재요원_이해도"
                ],
                "기본정보_시설정보_변경사항": [
                    "기본정보_변경여부",
                    "시설정보_변경여부",
                    "변경내역_관리대장_반영여부"
                ]
            }
        }

    def _load_review_patterns(self):
        """과거 보완요청 패턴 분석"""
        patterns = {
            "자체점검_내부자만_구성": {
                "키워드": ["자체점검이 내부자만으로 구성", "외부 전문가 부족"],
                "보완요청": "외부 전문가를 포함한 자체점검반 구성 필요"
            },
            "증빙자료_부족": {
                "키워드": ["증빙자료 부족", "확인 불가", "자료 미제출"],
                "보완요청": "개선사항 조치에 대한 증빙자료 제출 필요"
            },
            "체크리스트_미흡": {
                "키워드": ["체크리스트에 일부 X 표기", "해당없음 표기"],
                "보완요청": "체크리스트 항목별 정확한 점검 및 표기 필요"
            },
            "점검기간_부족": {
                "키워드": ["1일간 실시", "점검기간이 짧음"],
                "보완요청": "충분한 점검기간을 확보하여 상세한 점검 실시 필요"
            }
        }
        return patterns

    def parse_submission_documents(self, file_paths):
        """제출 서류 파싱"""
        parsed_data = {}

        for file_path in file_paths:
            if file_path.endswith('.xlsx'):
                parsed_data.update(self._parse_excel(file_path))
            elif file_path.endswith('.docx'):
                parsed_data.update(self._parse_word(file_path))
            elif file_path.endswith('.pdf'):
                parsed_data.update(self._parse_pdf(file_path))

        return parsed_data

    def _parse_excel(self, file_path):
        """엑셀 파일 파싱 - 체크리스트 분석"""
        df = pd.read_excel(file_path)

        checklist_results = {}
        improvement_items = []

        # 체크리스트 결과 추출
        for idx, row in df.iterrows():
            if '세부항목' in df.columns and '확인' in df.columns:
                item = row.get('세부항목', '')
                result = row.get('확인', '')
                checklist_results[item] = result

        # 개선사항 추출
        if '개선사항' in df.columns:
            improvement_items = df['개선사항'].dropna().tolist()

        return {
            'checklist_results': checklist_results,
            'improvement_items': improvement_items,
            'total_improvements': len(improvement_items)
        }

    def _parse_word(self, file_path):
        """워드 파일 파싱 - 자체점검 결과서"""
        doc = Document(file_path)

        inspection_info = {}
        inspection_team = []

        # 테이블에서 정보 추출
        for table in doc.tables:
            for row in table.rows:
                cells = [cell.text.strip() for cell in row.cells]
                if len(cells) >= 2:
                    if '자체점검반' in cells[0]:
                        inspection_team.append(cells[1])
                    elif '자체점검 일자' in cells[0]:
                        inspection_info['inspection_date'] = cells[1]

        return {
            'inspection_info': inspection_info,
            'inspection_team': inspection_team
        }


In [10]:
class KananaReviewSystem:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")
        self.model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # 과거 심사 데이터 로드
        self.historical_data = self._load_historical_review_data()
        self.review_criteria = self._define_review_criteria()

    def _load_historical_review_data(self):
        """과거 심사 데이터 로드 및 전처리"""
        # 첨부파일3의 데이터를 기반으로 학습 데이터 구성
        historical_reviews = [
            {
                "주요_점검내용": "화학사고 사후조치 대비 환경복원업체 점검",
                "우수내용": "환경복원 전문업체(4곳) 사전 계약체결을 통해 사고복구 계획 개선",
                "부족한_내용": "없음",
                "개선사항_건수": 3,
                "보완요청": "없음"
            },
            {
                "주요_점검내용": "자체점검 인터뷰(사업장, 취급자 등 일체)",
                "우수내용": "자체점검 결과에 대한 보완내역 사진대지 등 개선전후 확인이 가능하도록 제출",
                "부족한_내용": "자체점검 인터뷰 등 실제 점검 사진에 대한 증빙 필요",
                "개선사항_건수": 2,
                "보완요청": "증빙자료 제출 필요"
            }
        ]
        return historical_reviews

    def _define_review_criteria(self):
        """심사 기준 정의"""
        return {
            "필수_제출서류": [
                "자체점검_결과서",
                "자체점검_체크리스트",
                "개선사항_조치내역서",
                "변경내역_관리대장"
            ],
            "점검_품질_기준": {
                "최소_점검기간": 2,  # 일
                "외부전문가_포함": True,
                "증빙자료_첨부": True,
                "체크리스트_완성도": 0.9
            },
            "자동_보완요청_조건": [
                "자체점검이 내부자만으로 구성·실시",
                "연 단위 자체검검 계획 및 실시가 필요",
                "체크리스트에 일부 X 표기",
                "증빙자료 부족",
                "점검 소요 기간이 1일"
            ]
        }

    def analyze_submission(self, parsed_data):
        """제출 서류 종합 분석"""
        analysis_result = {
            "completeness_score": 0,
            "quality_score": 0,
            "identified_issues": [],
            "recommendation": "",
            "requires_supplement": False
        }

        # 1. 완성도 평가
        completeness_score = self._evaluate_completeness(parsed_data)
        analysis_result["completeness_score"] = completeness_score

        # 2. 품질 평가
        quality_score = self._evaluate_quality(parsed_data)
        analysis_result["quality_score"] = quality_score

        # 3. 이슈 식별
        issues = self._identify_issues(parsed_data)
        analysis_result["identified_issues"] = issues

        # 4. AI 기반 종합 판정
        recommendation = self._generate_ai_recommendation(parsed_data, issues)
        analysis_result["recommendation"] = recommendation

        # 5. 보완 요청 여부 결정
        analysis_result["requires_supplement"] = len(issues) > 0 or quality_score < 0.7

        return analysis_result

    def _evaluate_completeness(self, parsed_data):
        """서류 완성도 평가"""
        required_items = self.review_criteria["필수_제출서류"]
        submitted_items = list(parsed_data.keys())

        completion_rate = len(set(submitted_items) & set(required_items)) / len(required_items)
        return completion_rate

    def _evaluate_quality(self, parsed_data):
        """점검 품질 평가"""
        quality_factors = []

        # 점검팀 구성 평가
        inspection_team = parsed_data.get('inspection_team', [])
        if len(inspection_team) > 3:
            quality_factors.append(0.8)
        else:
            quality_factors.append(0.4)

        # 개선사항 수량 평가
        improvement_count = parsed_data.get('total_improvements', 0)
        if improvement_count >= 5:
            quality_factors.append(0.9)
        elif improvement_count >= 2:
            quality_factors.append(0.7)
        else:
            quality_factors.append(0.3)

        # 체크리스트 완성도
        checklist_results = parsed_data.get('checklist_results', {})
        if checklist_results:
            completed_items = sum(1 for v in checklist_results.values() if v == '적합')
            total_items = len(checklist_results)
            checklist_score = completed_items / total_items if total_items > 0 else 0
            quality_factors.append(checklist_score)

        return np.mean(quality_factors) if quality_factors else 0

    def _identify_issues(self, parsed_data):
        """이슈 식별"""
        issues = []

        # 자체점검팀 구성 이슈
        inspection_team = parsed_data.get('inspection_team', [])
        if len(inspection_team) <= 3:
            issues.append({
                "category": "점검팀_구성",
                "description": "자체점검이 내부자만으로 구성·실시",
                "severity": "중요"
            })

        # 개선사항 부족
        improvement_count = parsed_data.get('total_improvements', 0)
        if improvement_count < 2:
            issues.append({
                "category": "개선사항",
                "description": "개선사항 발굴 및 조치 내역 부족",
                "severity": "보통"
            })

        # 체크리스트 미완성
        checklist_results = parsed_data.get('checklist_results', {})
        incomplete_items = [k for k, v in checklist_results.items() if v in ['X', '미흡', '']]
        if len(incomplete_items) > 0:
            issues.append({
                "category": "체크리스트",
                "description": f"체크리스트 {len(incomplete_items)}개 항목 미완성",
                "severity": "중요"
            })

        return issues

    def _generate_ai_recommendation(self, parsed_data, issues):
        """AI 기반 종합 의견 생성"""
        prompt = f"""
        화학사고예방관리계획서 서면점검 결과를 분석하여 종합 의견을 제시해주세요.

        제출 현황:
        - 개선사항 건수: {parsed_data.get('total_improvements', 0)}건
        - 점검팀 구성: {len(parsed_data.get('inspection_team', []))}명

        식별된 이슈:
        {[issue['description'] for issue in issues]}

        다음 기준으로 종합 의견을 작성해주세요:
        1. 제출 서류의 적정성
        2. 자체점검의 충실성
        3. 개선 필요사항
        4. 최종 권고사항

        종합 의견:
        """

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=inputs['input_ids'].shape[1] + 200,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        recommendation = response.split("종합 의견:")[-1].strip()

        return recommendation


공문 자동 생성 시스템

In [11]:
class OfficialDocumentGenerator:
    def __init__(self):
        self.template_path = "templates/supplement_request_template.docx"

    def generate_supplement_request(self, company_info, analysis_result, output_path):
        """수정보완요청 공문 생성"""

        # 템플릿 문서 로드
        doc = Document()

        # 문서 헤더 설정
        self._add_header(doc)

        # 기본 정보 테이블
        self._add_company_info_table(doc, company_info)

        # 수정보완 사항 작성
        self._add_supplement_details(doc, analysis_result)

        # 문서 저장
        doc.save(output_path)

        return output_path

    def _add_header(self, doc):
        """공문 헤더 추가"""
        # 제목
        title = doc.add_heading('화학물질안전원', 0)
        title.alignment = 1  # 가운데 정렬

        # 부제목
        subtitle = doc.add_heading('2025년 2분기 화학사고예방관리계획서 서면점검 수정·보완 요청', level=1)

        # 날짜
        date_paragraph = doc.add_paragraph()
        date_paragraph.add_run(f"작성일: {datetime.now().strftime('%Y년 %m월 %d일')}")

    def _add_company_info_table(self, doc, company_info):
        """사업장 정보 테이블 추가"""
        table = doc.add_table(rows=4, cols=2)
        table.style = 'Table Grid'

        # 테이블 내용
        table_data = [
            ['상호(명칭)', company_info.get('company_name', '')],
            ['성명(대표자)', company_info.get('representative', '')],
            ['사업자등록번호', company_info.get('business_number', '')],
            ['주소', company_info.get('address', '')]
        ]

        for i, (key, value) in enumerate(table_data):
            table.cell(i, 0).text = key
            table.cell(i, 1).text = value

    def _add_supplement_details(self, doc, analysis_result):
        """수정보완 사항 상세 내용 추가"""
        doc.add_heading('수정·보완 사항', level=2)

        issues = analysis_result.get('identified_issues', [])

        if not issues:
            doc.add_paragraph("검토 결과 특별한 보완사항이 없어 완료 처리됩니다.")
            return

        # 보완 요청 사항 분류
        supplement_categories = {
            "필수자료": [],
            "그외자료": [],
            "기타": []
        }

        for issue in issues:
            category = issue.get('category', '')
            description = issue.get('description', '')

            if category in ['점검팀_구성', '체크리스트']:
                supplement_categories["필수자료"].append(description)
            elif category in ['개선사항', '증빙자료']:
                supplement_categories["그외자료"].append(description)
            else:
                supplement_categories["기타"].append(description)

        # 체크박스 형태로 보완사항 표시
        checkbox_table = doc.add_table(rows=1, cols=2)
        checkbox_table.style = 'Table Grid'

        checkbox_content = []
        if supplement_categories["필수자료"]:
            checkbox_content.append("☑ 필수자료(자체점검 결과서, 자체점검표, 개선사항 조치 내역서, 변경내역 관리대장)")
        else:
            checkbox_content.append("☐ 필수자료(자체점검 결과서, 자체점검표, 개선사항 조치 내역서, 변경내역 관리대장)")

        if supplement_categories["그외자료"]:
            checkbox_content.append("☑ 그 외 자료(개선사항 추가 발굴, 증빙자료 등)")
        else:
            checkbox_content.append("☐ 그 외 자료(개선사항 추가 발굴, 증빙자료 등)")

        checkbox_table.cell(0, 0).text = "수정·보완 사항(내역)"
        checkbox_table.cell(0, 1).text = "\n".join(checkbox_content)

        # 상세 보완 요청 사항
        doc.add_heading('상세 보완 요청 사항', level=3)

        for i, issue in enumerate(issues, 1):
            p = doc.add_paragraph()
            p.add_run(f"{i}. ").bold = True
            p.add_run(f"{issue['description']} (중요도: {issue['severity']})")

        # 제출 기한
        deadline = (datetime.now() + pd.Timedelta(days=10)).strftime('%Y년 %m월 %d일')
        doc.add_paragraph(f"\n위 사항에 대해 {deadline}까지 수정·보완하여 제출하시기 바랍니다.")

    def generate_completion_notice(self, company_info, output_path):
        """완료 통지서 생성"""
        doc = Document()

        # 헤더
        title = doc.add_heading('화학사고예방관리계획서 서면점검 완료 통지', 0)
        title.alignment = 1

        # 내용
        doc.add_paragraph(f"귀하가 제출한 화학사고예방관리계획서 서면점검 결과를 검토한 결과, 적정하게 작성되어 완료 처리됩니다.")

        # 사업장 정보
        table = doc.add_table(rows=2, cols=2)
        table.style = 'Table Grid'
        table.cell(0, 0).text = '상호(명칭)'
        table.cell(0, 1).text = company_info.get('company_name', '')
        table.cell(1, 0).text = '완료일자'
        table.cell(1, 1).text = datetime.now().strftime('%Y년 %m월 %d일')

        doc.save(output_path)
        return output_path


파일 업로드

In [23]:
from google.colab import files
uploaded = files.upload()

Saving rev detail.xlsx to rev detail (1).xlsx


In [14]:
from google.colab import files
uploaded = files.upload()

Saving company template.docx to company template.docx


엑셀 파일 학습

In [15]:
from docx import Document

# 1) 템플릿 로드
from docx import Document
doc = Document('/content/company template.docx')

# 2) 단락 정보 파싱
parsed_paragraphs = {p.text.split(":",1)[0]: p.text.split(":",1)[1].strip()
                     for p in doc.paragraphs if ":" in p.text}

# 3) 표(table) 정보 파싱
tables_data = []
for table in doc.tables:
    rows = [[cell.text for cell in row.cells] for row in table.rows]
    tables_data.append(rows)


pandas 로 엑셀 파싱

In [25]:
import pandas as pd
df = pd.read_excel('/content/rev detail (1).xlsx')
print(df.columns.tolist())


['구분', '심사자', '민원번호', '사업장명', '주요점검내용', '우수내용', '부족한내용', '자체점검 결과 \n개선사항 조치 내역 건수\n(숫자만 기재)', '사업장 담당자\n(변경시 기재 바람, \n검토결과 알림 송부 용)', '사업장 담당자 메일주소\n(변경시 기재 바람, \n검토결과 알림 송부 용)', '사업장등록번호', '주소지', '사업장 담당자', '사업장 담당자 연락처(내선)', '사업장 담당자 연락처(휴대폰)', '담당자 메일주소']


In [26]:
import pandas as pd
df = pd.read_excel('/content/rev detail (1).xlsx')

# 실제 존재하는 컬럼명 사용
df = df[['주요점검내용', '우수내용', '부족한내용']].dropna()


In [28]:
with open('train.jsonl','w',encoding='utf-8') as f:
    for _, row in df.iterrows():
        src = f"점검내용: {row['주요점검내용']} 부족: {row['부족한내용']} 우수: {row['우수내용']}"
        tgt = row['보완요청']
        f.write(json.dumps({"prompt": src, "completion": tgt}, ensure_ascii=False) + '\n')


KeyError: '보완요청'